In [ ]:
import sys

path='/home/tomas/Ulmer-Berechnung/alps2qutipplus-april/alps2qutipplus-main/'

sys.path.insert(1, path) 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import qutip as qutip
import scipy.linalg as linalg
import time
from multiprocessing import Pool
from itertools import product
from typing import Optional

import alpsqutip.parallelized_functions_and_workers as plME

from alpsqutip import (build_system, list_models_in_alps_xml,
                       list_geometries_in_alps_xml, graph_from_alps_xml,
                       model_from_alps_xml,
                       restricted_maxent_toolkit as me)

from alpsqutip.operators.states.utils import safe_exp_and_normalize ## function used to safely and robustly map K-states to states

from alpsqutip.scalarprod import fetch_covar_scalar_product
from alpsqutip.operators.states.gibbs import GibbsDensityOperator, GibbsProductDensityOperator

In [ ]:

from alpsqutip.operators.arithmetic import (
    ScalarOperator,
    LocalOperator,
    OneBodyOperator,
    Operator,
    ProductOperator,
    ScalarOperator,
    SumOperator,
    QutipOperator
)

from alpsqutip.operators.simplify import simplify_sum_operator
from alpsqutip.optimized_projections import opt_project_to_n_body_operator
from alpsqutip.operators.states.meanfield.projections import project_to_n_body_operator

In [ ]:
params={}

params['size']=7
params['Jx']=1.25; params['Jy'] = 0.8*params['Jx']; params['Jz']=0.*params['Jx']

from scipy.optimize import root, fsolve
Ffactor=np.real(max(np.roots(np.poly1d([1, 0, -(params['Jx']*params['Jy']+params['Jx']*params['Jy']+params['Jy']*params['Jz']), 
                           -2*params['Jx']*params['Jy']*params['Jz']]))))
chi_y=fsolve(lambda x,y: x*np.arcsinh(x)-np.sqrt(x**2+1)-y, 1e-1, args=(0))[0]
vLR=4*Ffactor*chi_y


In [ ]:
from itertools import combinations
from functools import reduce

# Build system
system = build_system(geometry_name="open chain lattice", model_name="spin", 
                      L=params['size'], J=1)

sites = [s for s in system.sites]
sx_ops = [system.site_operator("Sx", f'1[{i}]') for i in range(len(sites))]
sy_ops = [system.site_operator("Sy", f'1[{i}]') for i in range(len(sites))]
sz_ops = [system.site_operator("Sz", f'1[{i}]') for i in range(len(sites))]

# Optional: full identity operator
idop = reduce(Operator.__mul__, [system.site_operator('identity@1[' + str(i) + ']') for i in range(len(sites))])

# XYZ nearest-neighbor Hamiltonian
H_XYZ = 0
for i in range(len(sites) - 1):  # Only nearest-neighbors
    H_XYZ += (
        params['Jx'] * sx_ops[i] * sx_ops[i+1] +
        params['Jy'] * sy_ops[i] * sy_ops[i+1] +
        params['Jz'] * sz_ops[i] * sz_ops[i+1]
    )

H_XYZ = H_XYZ.simplify()
H = H_XYZ

In [ ]:
HBB0=[idop, system.site_operator('Sx', '1[0]'), system.site_operator('Sy', '1[0]'), system.site_operator('Sz', '1[0]')]

phi0 = np.array([.0, .25, .25, 10.])
K0 = phi0@HBB0
sigma0 = GibbsProductDensityOperator(K0)
phi0[0] = np.log(sigma0.tr())
K0 = phi0@HBB0
sigma0 = GibbsProductDensityOperator(K0)
[(sigma0 * op).tr() for op in sz_ops] 

In [ ]:
obs_SzA=sum(sz for sz in sz_ops)

timespan=np.linspace(.0, 70.1/vLR,100)

In [ ]:
evs_obs_ex=qutip.mesolve(H=H.to_qutip(), rho0=sigma0.to_qutip()
                         , tlist=timespan, e_ops=[obs_SzA.to_qutip()])

In [ ]:
plt.plot(timespan, evs_obs_ex.expect[0])

In [ ]:
simulations={}

In [ ]:

tgt_obs = obs_SzA
sp_local = fetch_covar_scalar_product(sigma0)

from scipy.linalg import cho_solve, cho_factor
       
simulations[1] = {
    "parms":{
    "chosen_depth": 6,
    "m0": 2,
    "ell_prime": None,
    "eps": 1e-4,
    },
  "saved_cut_times_index_ell":[0],
   "no_acts_ell" : [0],
    "local_bound_error_ell":[],
    "spectral_norm_Hij_tensor_ell":[],
    "instantaneous_w_errors":[],
} 

current_simulation = simulations[1]
current_simulation_parms = current_simulation["parms"]
chosen_depth = current_simulation_parms["chosen_depth"]
eps_tol = current_simulation_parms["eps"]
m0 = current_simulation_parms["m0"]
ell_prime=current_simulation_parms["ell_prime"]
num_workers=2

In [ ]:
print("a. start t=", list(timespan).index(0))
start = time.time()
local_bound_error_ell = []
simulations[1]["evs"] = [(sigma0* tgt_obs).tr()]  # initial observable expectation
sp_local = fetch_covar_scalar_product(sigma=sigma0)
sigma_act = sigma0
local_t_value = 0.0

# Initial bass construction and orthogonalization
HBB_ell_act = plME.parallelized_real_time_projection_of_hierarchical_basis(
    generator=H,
    seed_op=K0+1e-10*H,
    sigma_ref=sigma_act,
    nmax=m0,
    deep=chosen_depth,
    ell_prime=ell_prime,
    num_workers=num_workers
)

HBB_ell_act = [op.tidyup(1e-7) for op in HBB_ell_act ]
print(f"b. HBB_ell_act, {time.time() - start:.6f}s"); start = time.time()

Gram_matrix_act = plME.parallel_gram_matrix_fine(basis = HBB_ell_act, 
                                           sp = sp_local,
                                           num_workers=num_workers)

R_act = np.linalg.cholesky(Gram_matrix_act).conj().T

b_orth= linalg.inv(R_act.T) @ HBB_ell_act
print(f"c. orth basis act, {time.time() - start:.6f}s"); start = time.time()

Hij_tensor_act_non_orth = plME.compute_Hij_tensor_non_orth(
    basis = HBB_ell_act, 
    generator = H, 
    sp = sp_local,
    sigma_ref = sigma0,
    nmax=2,
    Gram = Gram_matrix_act,
    num_workers=num_workers
)

Hij_tensor_act = (linalg.inv(R_act).T 
                  @ Hij_tensor_act_non_orth 
                  @ linalg.inv(R_act)) ### In this convention for R_act, the orthonormal Hij_orth is
                                       ### Hij_orth = (R**-1).T @ Hij_non_orth @ (R**-1)

phi0_proj_act = np.array([sp_local(K0, op) for op in b_orth])
new_obs_local=sum(phi0a * opa for phi0a, opa in zip(phi0_proj_act, b_orth)).simplify().tidyup(1e-7)
print("Fita: Check", ((new_obs_local.to_qutip() - K0.to_qutip())).norm())
print(f"d. Hij-tensor-act, {time.time() - start:.6f}s"); start = time.time()

for t in timespan[1:]:
    start = time.time()
    
    delta_t = t - local_t_value
    exp_H = linalg.expm(delta_t * Hij_tensor_act)
    phi_local = np.real(exp_H @ phi0_proj_act)
    
    print(f"A. phi-local {time.time() - start:.2f}s");    start = time.time()
    
    K_local = (phi_local @ b_orth).simplify().tidyup(1e-7)
    #b_orth=None; HBB_ell_act = None
    
    print(f"B. obs-local {time.time() - start:.2f}s");    start = time.time()
        
    simulations[1]["evs"].append((GibbsDensityOperator(K_local)*tgt_obs).tr()) ### the minus is here
    
    print(f"C. Ev {time.time() - start:.2f}s");    start = time.time()
    
    # Compute the local bound error ratio
    numerator = me.m_th_partial_sum(phi=phi_local, m=2)
    denominator = me.m_th_partial_sum(phi=phi_local, m=0)
    local_bound_error_ell.append(numerator / denominator)
    
    # Check if error is above threshold, trigger basis update
    if abs(local_bound_error_ell[-1]) >= eps_tol:
        local_t_value = t
        print(f"a. Renormalization at t={list(timespan).index(t)}")
        
        print("a. start t=", list(timespan).index(t))
        if False:
            K_act=plME.general_worker(("projection",(K_local + 1e-10*H,
                                                     1, sigma_act)))
            sigma_act=GibbsProductDensityOperator(K_act)
            sp_local = fetch_covar_scalar_product(sigma_act)
        
        start = time.time()
        HBB_ell_act = plME.parallelized_real_time_projection_of_hierarchical_basis(
                generator=H,
                seed_op=K_local,
                sigma_ref=sigma_act,
                nmax=m0,
                deep=chosen_depth,
                ell_prime=None,
                num_workers=num_workers
        )
        print(f"b. HBB_ell_act, {time.time() - start:.6f}s")
        
        start = time.time()
        
        Gram_matrix_act = plME.parallel_gram_matrix_fine(basis = HBB_ell_act, 
                                           sp = sp_local,
                                           num_workers=num_workers)
        
        R_act = np.linalg.cholesky(Gram_matrix_act).conj().T

        b_orth= linalg.inv(R_act.T) @ HBB_ell_act

        print(f"c. orth basis act, {time.time() - start:.6f}s")
        
        start = time.time()
        
        Hij_tensor_act_non_orth = plME.compute_Hij_tensor_non_orth(
            basis = HBB_ell_act, 
            generator = H, 
            sp = sp_local,
            sigma_ref = sigma_act,
            nmax=m0,
            Gram = Gram_matrix_act,
            num_workers=num_workers+1)
            
        print(f"d. Hij-tensor-act, {time.time() - start:.6f}s")
        Hij_tensor_act = linalg.inv(R_act).T @ Hij_tensor_act_non_orth @ linalg.inv(R_act)
        phi0_proj_act = np.array([sp_local(K_local, op) for op in b_orth])
        new_obs_local=sum(phi0a * opa for phi0a, opa in zip(phi0_proj_act, b_orth)).simplify()
        print("Fita: Check", ((K_local.to_qutip() - new_obs_local.to_qutip())).norm())
    else:
        # If error below tolerance, keep current basis and coefficients
        pass

In [ ]:
tgt_obs = obs_SzA
sp_local = fetch_covar_scalar_product(sigma0)

from scipy.linalg import cho_solve, cho_factor
       
simulations[2] = {
    "parms":{
    "chosen_depth": 6,
    "m0": 3,
    "ell_prime": None,
    "eps": 1e-4,
    },
  "saved_cut_times_index_ell":[0],
   "no_acts_ell" : [0],
    "local_bound_error_ell":[],
    "spectral_norm_Hij_tensor_ell":[],
    "instantaneous_w_errors":[],
} 

current_simulation = simulations[2]
current_simulation_parms = current_simulation["parms"]
chosen_depth = current_simulation_parms["chosen_depth"]
eps_tol = current_simulation_parms["eps"]
m0 = current_simulation_parms["m0"]
ell_prime=current_simulation_parms["ell_prime"]
num_workers=3

In [ ]:
print("a. start t=", list(timespan).index(0))
start = time.time()
local_bound_error_ell = []
simulations[2]["evs"] = [(sigma0* tgt_obs).tr()]  # initial observable expectation
sp_local = fetch_covar_scalar_product(sigma=sigma0)
sigma_act = sigma0
local_t_value = 0.0

# Initial bass construction and orthogonalization
HBB_ell_act = plME.parallelized_real_time_projection_of_hierarchical_basis(
    generator=H,
    seed_op=K0+1e-10*H,
    sigma_ref=sigma_act,
    nmax=m0,
    deep=chosen_depth,
    ell_prime=ell_prime,
    num_workers=num_workers
)

HBB_ell_act = [op.tidyup(1e-7) for op in HBB_ell_act ]
print(f"b. HBB_ell_act, {time.time() - start:.6f}s"); start = time.time()

Gram_matrix_act = plME.parallel_gram_matrix_fine(basis = HBB_ell_act, 
                                           sp = sp_local,
                                           num_workers=num_workers)

R_act = np.linalg.cholesky(Gram_matrix_act).conj().T

b_orth= linalg.inv(R_act.T) @ HBB_ell_act
print(f"c. orth basis act, {time.time() - start:.6f}s"); start = time.time()

Hij_tensor_act_non_orth = plME.compute_Hij_tensor_non_orth(
    basis = HBB_ell_act, 
    generator = H, 
    sp = sp_local,
    sigma_ref = sigma0,
    nmax=2,
    Gram = Gram_matrix_act,
    num_workers=num_workers
)

Hij_tensor_act = (linalg.inv(R_act).T 
                  @ Hij_tensor_act_non_orth 
                  @ linalg.inv(R_act)) ### In this convention for R_act, the orthonormal Hij_orth is
                                       ### Hij_orth = (R**-1).T @ Hij_non_orth @ (R**-1)

phi0_proj_act = np.array([sp_local(K0, op) for op in b_orth])
new_obs_local=sum(phi0a * opa for phi0a, opa in zip(phi0_proj_act, b_orth)).simplify().tidyup(1e-7)
print("Fita: Check", ((new_obs_local.to_qutip() - K0.to_qutip())).norm())
print(f"d. Hij-tensor-act, {time.time() - start:.6f}s"); start = time.time()

for t in timespan[1:]:
    start = time.time()
    
    delta_t = t - local_t_value
    exp_H = linalg.expm(delta_t * Hij_tensor_act)
    phi_local = np.real(exp_H @ phi0_proj_act)
    
    print(f"A. phi-local {time.time() - start:.2f}s");    start = time.time()
    
    K_local = (phi_local @ b_orth).simplify().tidyup(1e-7)
    #b_orth=None; HBB_ell_act = None
    
    print(f"B. obs-local {time.time() - start:.2f}s");    start = time.time()
        
    simulations[2]["evs"].append((GibbsDensityOperator(K_local)*tgt_obs).tr()) ### the minus is here
    
    print(f"C. Ev {time.time() - start:.2f}s");    start = time.time()
    
    # Compute the local bound error ratio
    numerator = me.m_th_partial_sum(phi=phi_local, m=2)
    denominator = me.m_th_partial_sum(phi=phi_local, m=0)
    local_bound_error_ell.append(numerator / denominator)
    
    # Check if error is above threshold, trigger basis update
    if abs(local_bound_error_ell[-1]) >= eps_tol:
        local_t_value = t
        print(f"a. Renormalization at t={list(timespan).index(t)}")
        
        print("a. start t=", list(timespan).index(t))
        if False:
            K_act=plME.general_worker(("projection",(K_local + 1e-10*H,
                                                     1, sigma_act)))
            sigma_act=GibbsProductDensityOperator(K_act)
            sp_local = fetch_covar_scalar_product(sigma_act)
        
        start = time.time()
        HBB_ell_act = plME.parallelized_real_time_projection_of_hierarchical_basis(
                generator=H,
                seed_op=K_local,
                sigma_ref=sigma_act,
                nmax=m0,
                deep=chosen_depth,
                ell_prime=None,
                num_workers=num_workers
        )
        print(f"b. HBB_ell_act, {time.time() - start:.6f}s")
        
        start = time.time()
        
        Gram_matrix_act = plME.parallel_gram_matrix_fine(basis = HBB_ell_act, 
                                           sp = sp_local,
                                           num_workers=num_workers)
        
        R_act = np.linalg.cholesky(Gram_matrix_act).conj().T

        b_orth= linalg.inv(R_act.T) @ HBB_ell_act

        print(f"c. orth basis act, {time.time() - start:.6f}s")
        
        start = time.time()
        
        Hij_tensor_act_non_orth = plME.compute_Hij_tensor_non_orth(
            basis = HBB_ell_act, 
            generator = H, 
            sp = sp_local,
            sigma_ref = sigma_act,
            nmax=m0,
            Gram = Gram_matrix_act,
            num_workers=num_workers)
            
        print(f"d. Hij-tensor-act, {time.time() - start:.6f}s")
        Hij_tensor_act = linalg.inv(R_act).T @ Hij_tensor_act_non_orth @ linalg.inv(R_act)
        phi0_proj_act = np.array([sp_local(K_local, op) for op in b_orth])
        new_obs_local=sum(phi0a * opa for phi0a, opa in zip(phi0_proj_act, b_orth)).simplify()
        print("Fita: Check", ((K_local.to_qutip() - new_obs_local.to_qutip())).norm())
    else:
        # If error below tolerance, keep current basis and coefficients
        pass